In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append("..")

import csv
import os

from evaluator import Evaluator

from config import OPUS_MT_MODELS, prefixes
from utils.data_utils import get_bbt_data
from utils.model_utils import load_baseline, load_model, translate_all

Loading config. model overview: {'no': 'Helsinki-NLP/opus-mt-en-gmq', 'is': 'Helsinki-NLP/opus-mt-en-gmq', 'lt': 'Helsinki-NLP/opus-mt-en-bat', 'pl': 'Helsinki-NLP/opus-mt-en-zlw', 'hu': 'Helsinki-NLP/opus-mt-en-urj', 'fr': 'Helsinki-NLP/opus-mt-en-roa', 'ms': 'Helsinki-NLP/opus-mt-en-map', 'sq': 'Helsinki-NLP/opus-mt-en-sq', 'eu': 'Helsinki-NLP/opus-mt-en-eu'}


In [2]:
small_subset = [
    "why would Amy feel the need to lie to me?",
    "I am not gonna spend weeks in a library looking up old papers can't we hire a grad-student to do it?",
    "well what if we ask one of our friends to help us out how about Leonard?",
    "It's kinda beautiful isn't it?",
]

In [5]:
lang = "no"
src_texts, tar_texts, prefixed = get_bbt_data(prefix=prefixes.get(lang, ""))

subset_map = {}
for src, tar in zip(src_texts, tar_texts):
    if src in small_subset:
        subset_map[src] = tar
subset_map
print(subset_map)

# prefixed = [">>nob<< " + text for text in small_subset]
# prefixed


{'why would Amy feel the need to lie to me?': 'Hvorfor løy Amy til meg?', "I am not gonna spend weeks in a library looking up old papers can't we hire a grad-student to do it?": 'Kan vi ikke betale en doktorand for å sjekke sitatene?', 'well what if we ask one of our friends to help us out how about Leonard?': 'Vi kan be Leonard om hjelp.', "It's kinda beautiful isn't it?": 'Det er jo vakkert.'}


In [6]:
baseline_model, baseline_tokenizer = load_baseline(OPUS_MT_MODELS[lang])
baseline_translations = translate_all(prefixed, baseline_model, baseline_tokenizer, batch_size=16)
del baseline_model, baseline_tokenizer

100%|██████████| 6/6 [00:02<00:00,  2.85it/s]


In [8]:
baseline_translations[:3]

['Kan jeg fôre jordnøtter til elefantene på bryllupet ditt?',
 'Selvfølgelig vil det være elefanter det er en stereotype som du fôrer dem jordnøtter',
 'Takk Penny og Leonard Jeg håpet jeg kunne sitte ved siden av Anu']

In [9]:
# tar_texts = subset_map.values()
# src_texts = subset_map.keys()

evaluator = Evaluator()
baseline_metrics = evaluator.get_results(
    predictions=baseline_translations,
    references=tar_texts,
    source=src_texts,
    lang=lang,
    include_metrics=True
)

[nltk_data] Downloading package wordnet to /home/tollef/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/tollef/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tollef/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
84it [00:14,  5.66it/s]


In [24]:
import pandas as pd

def get_metric_df(metric_results, show_original_metrics=False):
    df = pd.DataFrame(metric_results).T
    df["metrics"] = df["metrics"].apply(lambda x: {k: v for k, v in x.items() if k not in ['length', 'len_ratio']})
    df = pd.concat([df.drop(['metrics'], axis=1), df['metrics'].apply(pd.Series)], axis=1)

    metrics = df.select_dtypes(include=['float64', 'int64']).columns
    originals = df[metrics]
    for metric in metrics:
        df[metric] = df[metric] / df[metric].max()

    df["score"] = df[metrics].sum(axis=1)
    df['score'] = df['score'].multiply(10).astype(int)
    
    df = df.drop(metrics, axis=1)

    if show_original_metrics:
        df = pd.concat([df, originals], axis=1)
    return df

get_metric_df(baseline_metrics, show_original_metrics=True)

,target,prediction,score,bleu,r1,r2,rl,chrF,chrf++,meteor,bert_f1
can I feed peanuts to the elephants at your wedding?,Får jeg gi elefantene peanøtter?,Kan jeg fôre jordnøtter til elefantene på bryllupet ditt?,36,5.52,0.44,0.00,0.22,47.23,40.94,0.23,0.78
of course there'll be elephants it's a stereotype that you feed them peanuts,"Jo, men de får ingen peanøtter.",Selvfølgelig vil det være elefanter det er en stereotype som du fôrer dem jordnøtter,17,0.00,0.15,0.00,0.15,20.61,15.46,0.00,0.67
thank you Penny and Leonard I was kind of hoping I could sit next to Anu,"Jeg vil sitte med Anu, Leonard.",Takk Penny og Leonard Jeg håpet jeg kunne sitte ved siden av Anu,32,4.37,0.40,0.00,0.30,33.01,30.37,0.24,0.77
yeah it's just we haven't done it yet and I think it's important to make sure we're compatible before we get married,Vi bør sjekke at vi passer sammen før vi gifter oss.,Ja det er bare vi ikke har gjort det ennå og jeg tror det er viktig å være sikker på at vi er ko...,42,9.94,0.38,0.25,0.38,34.08,34.02,0.49,0.79
it doesn't have to be tonight how about this weekend? I can get us a room at my hotel,Det må ikke skje i kveld. Jeg kan ordne rom på hotellet i helgen.,"Det trenger ikke være i kveld, hva med denne helgen? jeg kan skaffe oss et rom på hotellet mitt",51,9.85,0.59,0.25,0.53,47.65,44.72,0.50,0.80
"okay, sounds nice",Ja!,"OK, høres fint ut",7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.63
I am not worried if anything I'm overconfident,"Nei, heller for selvsikker.",Jeg er ikke bekymret hvis noe jeg er for tillitsfull,20,4.20,0.14,0.00,0.14,24.14,19.94,0.08,0.70
hey guys before Anu gets here can I talk about the seating situation,Kan vi snakke om sitteplassene?,Hei gutta før Anu kommer hit kan jeg snakke om sittestillingen,39,7.50,0.35,0.13,0.35,41.15,36.52,0.39,0.72
ah yes I wanted to get a jump on planning a day of fun for you,Jeg planlegger en artig dag for deg.,ah ja jeg ønsket å få et hopp på planlegging en dag av moro for deg,40,5.82,0.45,0.10,0.45,37.03,34.46,0.42,0.76
ah to be honest I'm pretty anxious I mean this is the woman I'm marrying what if it's no good?,Jeg er ganske nervøs. Vi skal jo gifte oss. Tenk om det ikke blir bra.,ah for å være ærlig Jeg er ganske engstelig Jeg mener dette er kvinnen jeg gifter meg med hva om...,39,9.80,0.35,0.21,0.35,37.66,35.50,0.31,0.75


In [14]:
baseline_map = {}
for _source, _results in baseline_metrics.items():
    baseline_map[_source] = _results["prediction"]
# baseline_map

In [15]:
def print_results(metrics, n=5):
    c = 0
    for _source, _results in metrics.items():
        print(f"Source: {_source}")
        print(f"Target: {_results['target']}")
        print(f"Pred: {_results['prediction']}")
        print(f"Metrics: {_results['metrics']}")
        print()
        c += 1
        if c >= n:
            break
    
# print_results(baseline_metrics)

# multisize

In [ ]:
model2_results = {}
show_sample_sizes = [5000, 25000, 100000, 250000]

from config import get_compression_prefix

included_compressions = [0.5, 0.7, 1.0]
MODEL_DIR = "../OUTPUT-flex-multisize-no-gmemodel"
for ckpt in sorted(os.listdir(MODEL_DIR), key=lambda x: int(x.split("-")[-1])):
    print(ckpt)

    model, tokenizer = load_model(
        path=f"{MODEL_DIR}/{ckpt}"
    )
    print(f"Model loaded: {MODEL_DIR}/{ckpt}")
    sample_sizes = int(ckpt.split("-")[-1])
    if sample_sizes not in show_sample_sizes:
        continue

    # for comp in COMPRESSION_RATIOS:
    for comp in included_compressions:
        prefix = get_compression_prefix(comp, lang)
        print(f"Compression ratio: {comp}, prefix: {prefix}")
        _prefixed = [f"{prefix} {text}" for text in src_texts]
        translated = translate_all(_prefixed, model, tokenizer, batch_size=16)
        _results = evaluator.get_results(
            predictions=translated,
            references=tar_texts,
            source=src_texts,
            lang=lang,
            include_metrics=True,
        )
        model2_results[(sample_sizes, comp)] = _results
    print()

model2_results

In [ ]:
import pandas as pd
from collections import defaultdict
from IPython.display import display


# included_compressions = [0.5, 0.7, 1.0]
interesting_sentences = [
    "can I feed peanuts to the elephants at your wedding?",
    "you realise we still have a lot of work to do before we can publish",
    "It's kinda beautiful isn't it?"
]
interesting_sentences = [sentence.lower() for sentence in interesting_sentences]

gathered_data = defaultdict(lambda: defaultdict(list))
for (sample_sizes, comp), results in model2_results.items():
    for _source, _results in results.items():
        pred = _results["prediction"]
        gathered_data[(_source, _results["target"])][sample_sizes].append((comp, pred))

dfs = []
for (source, target), data in gathered_data.items():
    # print(source)
    # print(target)
    # print(f"Baseline: {baseline_map[source]}")
    df = pd.DataFrame(data).T
    df.columns = included_compressions
    pd.set_option('display.max_colwidth', None)
    # now convert each text field from (number, text) to text
    for col in df.columns:
        df[col] = df[col].apply(lambda x: x[1])
    # add baseline:
    # df["baseline"] = baseline_map[source]
    dfs.append(df)
    # display(df)
    # print("_"*40)
df = pd.concat(dfs)
# df["sample"] = df.index
# sort the sample by 1k, 5k, 10k, etc
# print(df.to_latex())
# df = df.groupby("sample").agg(lambda x: "\\\\".join(x.tolist()))
# instead of a list, make it a newline of each sent
# df = df.applymap(lambda x: "\n\n".join(x))
# show newlines in the dataframe
pd.set_option('display.max_colwidth', 100)
# display(df.style.set_properties(**{ 'white-space': 'pre-wrap' }))

In [21]:
# display entire df
pd.set_option('display.max_rows', None)
df

,0.5,0.7,1.0
5000,Kan jeg gi jordnøtter til elefantene på bryllupet ditt?,Kan jeg gi jordnøtter til elefantene på bryllupet ditt?,Kan jeg gi jordnøtter til elefantene på bryllupet ditt?
25000,Kan jeg mate elefantene?,Kan jeg mate elefantene på bryllupet ditt?,Kan jeg gi elefantene jordnøtter på bryllupet ditt?
100000,Kan jeg gi elefantene peanøtter?,Kan jeg gi elefantene peanøtter?,Kan jeg gi elefantene peanøtter på bryllupet ditt?
250000,Kan jeg gi elefantene peanøtter?,Kan jeg mate elefantene peanøtter?,Kan jeg mate elefantene peanøtter i bryllupet ditt?
5000,Det er en stereotype du mater peanuts.,Det er en stereotype du mater peanuts.,Det er en stereotype som du mater peanuts.
25000,Det er en stereotype du mater peanøtter.,Det er en stereotype du mater peanøttene.,Det er en stereotype elefanter som du mater peanøtter.
100000,Det er en stereotype du mater peanøtter.,Det er en stereotype du mater peanøtter.,Selvsagt er det elefanter det er en stereotype du mater peanøtter
250000,Det er en stereotype du mater peanøtter.,Det er en stereotype du mater peanøtter.,Selvsagt er det elefanter det er en stereotype du mater peanøtter.
5000,"Takk, Penny og Leonard. Jeg håpet jeg kunne sitte ved siden av Anu.","Takk, Penny og Leonard. Jeg håpet jeg kunne sitte ved siden av Anu.",Takk Penny og Leonard jeg håpet jeg kunne sitte ved siden av Anu
25000,"Takk, Penny og Leonard.","Penny og Leonard, jeg håpet jeg kunne sitte ved siden av Anu.","Takk, Penny og Leonard, jeg håpet jeg kunne sitte ved siden av Anu."
